# Model Training for Real Time Communication through AI for Specially Abled

# Loading the Dataset & Image Data Generation

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install tensorflow==2.7.1

     |████████████████████████████████| 495.2 MB 38 kB/s /s eta 0:00:01                     | 44.8 MB 25.3 MB/s eta 0:00:18
     |████████████████████████████████| 14.1 MB 95.5 MB/s eta 0:00:01████████▎                      | 4.1 MB 95.5 MB/s eta 0:00:01
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.2
    Uninstalling tensorflow-2.7.2:
      Successfully uninstalled tensorflow-2.7.2


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-11-16 05:51:22.827830: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [4]:
# Training Datagen
train_datagen = ImageDataGenerator(rescale=1/255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)
# Testing Datagen
test_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
Wx5i1-Zv-rX46YIIPPDLrXiMMbCRcqnV3yg385HjlWVj

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='x3UYFhiwX3KxkxO9DjLrNlK3U1HZz1Z_6cuoxTfJtUg3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassificationmodel-donotdelete-pr-fpyzmkjffhgo06'
object_key = 'Dataset.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
# Unzip the Dataset Zip File
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
%%bash
ls Dataset

test_set
training_set


In [8]:
# Training Dataset
x_train=train_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/training_set',target_size=(64,64), class_mode='categorical',batch_size=900)
# Testing Dataset
x_test=test_datagen.flow_from_directory(r'/home/wsuser/work/Dataset/test_set',target_size=(64,64), class_mode='categorical',batch_size=900)

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


In [9]:
print("Len x-train : ", len(x_train))
print("Len x-test : ", len(x_test))

Len x-train :  18
Len x-test :  3


In [10]:
# The Class Indices in Training Dataset
x_train.class_indices

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8}

# Model Creation

In [11]:
# Importing Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense

In [12]:
# Creating Model
model=Sequential()

2022-11-16 06:15:16.298665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-16 06:15:16.298702: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [13]:
# Adding Layers
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

# Adding Hidden Layers
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

# Adding Output Layer
model.add(Dense(9,activation='softmax'))

In [14]:
# Compiling the Model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
# Fitting the Model Generator
model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

/tmp/wsuser/ipykernel_164/1042518445.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))


Epoch 1/10
18/18 [==============================] - 42s 2s/step - loss: 0.9667 - accuracy: 0.6716 - val_loss: 0.3323 - val_accuracy: 0.8867
Epoch 2/10
18/18 [==============================] - 40s 2s/step - loss: 0.2131 - accuracy: 0.9382 - val_loss: 0.1784 - val_accuracy: 0.9507
Epoch 3/10
18/18 [==============================] - 39s 2s/step - loss: 0.0849 - accuracy: 0.9771 - val_loss: 0.1533 - val_accuracy: 0.9676
Epoch 4/10
18/18 [==============================] - 40s 2s/step - loss: 0.0465 - accuracy: 0.9876 - val_loss: 0.1792 - val_accuracy: 0.9667
Epoch 5/10
18/18 [==============================] - 41s 2s/step - loss: 0.0315 - accuracy: 0.9917 - val_loss: 0.1641 - val_accuracy: 0.9600
Epoch 6/10
18/18 [==============================] - 40s 2s/step - loss: 0.0223 - accuracy: 0.9946 - val_loss: 0.1543 - val_accuracy: 0.9733
Epoch 7/10
18/18 [==============================] - 40s 2s/step - loss: 0.0128 - accuracy: 0.9971 - val_loss: 0.1310 - val_accuracy: 0.9773
Epoch 8/10
18/18 [==

# Saving the Model

In [16]:
model.save('deafanddumb.h5')
# Current accuracy is 0.8154

In [18]:
# Convert the Saved Model to a Tar Compressed Format
!tar -zcvf trainedModel.tgz deafanddumb.h5

deafanddumb.h5


In [19]:
%%bash
ls -ll

total 207072
drwxrwx--- 4 wsuser wscommon      4096 Nov 16 06:12 Dataset
-rw-rw---- 1 wsuser wscommon 111324760 Nov 16 06:24 deafanddumb.h5
-rw-rw---- 1 wsuser wscommon 100705415 Nov 16 06:25 trainedModel.tgz


# Watson Machine Learning

In [20]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 23.5 MB/s eta 0:00:01


In [21]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "riacBb11IauIqYeqyG0JXyv497ILBYpBK-i9ONnrRRO-"
}

client = APIClient(wml_credentials)

# Save to Deployment Space

In [22]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [24]:
space_uid = guid_from_space_name(client, 'model1')
print("Space UID : ", space_uid)

Space UID :  f334d222-b62c-4464-b436-31daf2a464bd


In [25]:
client.set.default_space(space_uid)

'SUCCESS'

In [26]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [27]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [28]:
model_details = client.repository.store_model(model='trainedModel.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_id(model_details)

In [29]:
model_id

'b541a7dd-164e-4153-a753-6d2a0a34962d'